In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [67]:
df = pd.DataFrame(columns=('title', 'dtg', 'dist', 'release', 'genre', 'runtime', 'budget', 'rating', 'foreign', 'foreign_dist', 'foreign_release', 'foreign_opening', 'foreign_pct', 'foreign_total', 'foreign_asof', 'hungary', 'hun_dist', 'hun_release', 'hun_opening', 'hun_pct', 'hun_total', 'hun_asof'))

In [44]:
list_of_urls = [
    'http://www.boxofficemojo.com/movies/?page=intl&id=easya.htm',
    'http://www.boxofficemojo.com/movies/?page=intl&id=child.htm'
]

In [50]:
def setup(site, movie_data):
    movie_title = ""
    movie_title = site.find('title').text.split(" (")[0]
    movie_data.append(movie_title)
    page_header = site.find_all('td')[4]
    international = site.find_all('table')[7]
    countries = international.find_all('tr')
    return movie_data, countries, page_header

In [51]:
def get_header_value(table, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = table.find(text=re.compile(field_name))
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return ''

In [52]:
def header(table, list_to_append):
    dtg = get_header_value(table,'Domestic Total')
    dist = get_header_value(table,'Distributor')
    release = get_header_value(table, 'Release Date')
    genre = get_header_value(table, 'Genre')
    runtime = get_header_value(table,'Runtime')
    budget = get_header_value(table, 'Budget')
    rating = get_header_value(table,'MPAA Rating')
    list_to_append.append(dtg)
    list_to_append.append(dist)
    list_to_append.append(release)
    list_to_append.append(genre)
    list_to_append.append(runtime)
    list_to_append.append(budget)
    list_to_append.append(rating)

In [61]:
# do this for foreign_total and for Hungary
# e.g. get_international('Hungary', hungary)
def get_international(country, country_list, country_table):
    country_data = ''
    for row in country_table:
        data = row.find_all('td')
        if data[0].find(text=country):
            country_data = row
    #get country name, insert as first element to list
    country_list.append(country_data.find('a').contents[0])
    columns = country_data.find_all('td')[1:]
    for column in columns:
        country_list.append(column.get_text())
    return country_list

In [62]:
def get_data(url):
    # return row of df if it exists
    # break out of that function but continue for loop if it does not
    response = requests.get(url)
    page = BeautifulSoup(response.text)
    
    movie_data = []
    foreign_total = []
    hungary = []
    
    if page.find_all(text="Hungary"):
        movie_data, countries, page_header = setup(page, movie_data)
        header(page_header, movie_data)
        get_international('FOREIGN TOTAL', foreign_total, countries)
        get_international('Hungary', hungary, countries)
        movie_row = movie_data + foreign_total + hungary
        return movie_row
        

In [68]:
df

,title,dtg,dist,release,genre,runtime,budget,rating,foreign,foreign_dist,...,foreign_pct,foreign_total,foreign_asof,hungary,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof


In [70]:
for url in list_of_urls:
    row = get_data(url)
    if row:
        row_s = pd.Series(row, index=df.columns)
        df = df.append(row_s, ignore_index=True)

In [71]:
df

,title,dtg,dist,release,genre,runtime,budget,rating,foreign,foreign_dist,...,foreign_pct,foreign_total,foreign_asof,hungary,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Easy A,"$58,401,464",Sony / Screen Gems,"September 17, 2010",Romantic Comedy,1 hrs. 32 min.,$8 million,PG-13,FOREIGN TOTAL,-,...,6.8%,"$16,550,841",3/20/11,Hungary,InterCom,10/28/10,"$47,154",40.6%,"$116,150",11/7/10
